1. Установка библиотек

In [ ]:
pip install transformers faiss-cpu sentence-transformers numpy scikit-learn torch

2.1. Подготовка данных и векторизация

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss

# Пример базы знаний (документы)
documents = [
    "Как сбросить пароль в системе?",
    "Где найти настройки VPN?",
    "Как установить обновление?",
    "Почему не работает интернет?",
    "Как создать новую учетную запись?"
]

# 1. Векторизация с помощью TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(documents).toarray()

# 2. Векторизация с помощью BERT
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
bert_vectors = model.encode(documents)

# 3. Создание FAISS-индекса для BERT-векторов
dim = bert_vectors.shape[1]
index = faiss.IndexFlatIP(dim)  # Используем косинусное сходство
index.add(bert_vectors.astype('float32'))


2.2. Поиск похожих документов

In [ ]:
def search_similar_documents(query, top_k=3, method='bert'):
    # Векторизация запроса
    if method == 'tfidf':
        query_vec = tfidf_vectorizer.transform([query]).toarray()
        # Сравнение с документами (косинусное сходство)
        similarities = np.dot(query_vec, tfidf_vectors.T)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
    elif method == 'bert':
        query_vec = model.encode([query])
        query_vec = query_vec.astype('float32')
        # Поиск в FAISS
        distances, top_indices = index.search(query_vec, top_k)

    # Возврат результатов
    results = [(documents[i], float(distances[0][j] if method == 'bert' else similarities[i]))
               for j, i in enumerate(top_indices)]
    return results

# Пример поиска
query = "Не могу войти в аккаунт"
results = search_similar_documents(query, top_k=2, method='bert')
print("Ближайшие документы:")
for doc, score in results:
    print(f"[Score: {score:.2f}] {doc}")


2.3. Генерация ответа (RAG)

In [ ]:
from transformers import pipeline

# Инициализация генеративной модели (ruGPT-3)
generator = pipeline(
    'text-generation',
    model='sberbank-ai/rugpt3large_based_on_gpt2'
)

def generate_answer(query, context):
    prompt = f"Вопрос: {query}\nКонтекст: {context}\nОтвет:"
    answer = generator(
        prompt,
        max_length=200,
        num_return_sequences=1,
        temperature=0.7
    )
    return answer[0]['generated_text'].split("Ответ:")[-1].strip()

# Пример генерации
context = " ".join([doc for doc, _ in results])
answer = generate_answer(query, context)
print("\nСгенерированный ответ:")
print(answer)


3. Интеграция всех компонентов (RAG Pipeline)

In [ ]:
class RAGSystem:
    def __init__(self):
        # Инициализация моделей
        self.tfidf_vectorizer = TfidfVectorizer()
        self.embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        self.generator = pipeline('text-generation', model='sberbank-ai/rugpt3large_based_on_gpt2')
        self.index = None
        self.documents = []

    def add_documents(self, docs):
        self.documents = docs
        # Обучение TF-IDF
        self.tfidf_vectors = self.tfidf_vectorizer.fit_transform(docs).toarray()
        # Векторизация BERT
        bert_vectors = self.embedding_model.encode(docs)
        # Создание FAISS-индекса
        dim = bert_vectors.shape[1]
        self.index = faiss.IndexFlatIP(dim)
        self.index.add(bert_vectors.astype('float32'))

    def search(self, query, top_k=3):
        query_vec = self.embedding_model.encode([query]).astype('float32')
        distances, indices = self.index.search(query_vec, top_k)
        return [(self.documents[i], float(distances[0][j])) for j, i in enumerate(indices[0])]

    def generate(self, query, context):
        prompt = f"Вопрос: {query}\nКонтекст: {context}\nОтвет:"
        answer = self.generator(prompt, max_length=200, temperature=0.7)[0]['generated_text']
        return answer.split("Ответ:")[-1].strip()

# Использование
rag = RAGSystem()
rag.add_documents(documents)
query = "Как восстановить доступ к системе?"
results = rag.search(query)
context = " ".join([doc for doc, _ in results])
answer = rag.generate(query, context)
print(answer)


4. Оптимизации и улучшения
1.	Для больших данных используйте FAISS-IVF или HNSW:


In [ ]:
quantizer = faiss.IndexFlatIP(dim)
index = faiss.IndexIVFFlat(quantizer, dim, 100)  # 100 кластеров
index.train(bert_vectors.astype('float32'))
index.add(bert_vectors.astype('float32'))


In [ ]:
2.	rag_model = pipeline('text2text-generation', model='facebook/rag-token-base')